# ⚖️ Concept 5: Weight Initialization Strategies

## Deep Neural Network Architectures - Week 5
**Module:** 2 - Optimization and Regularization  
**Topic:** Proper Weight Initialization for Gradient Flow

---

## 📋 Learning Objectives
By the end of this notebook, you will:
1. **Understand** why weight initialization is critical for gradient flow
2. **Implement** Xavier/Glorot and He initialization strategies
3. **Compare** different initialization methods experimentally
4. **Choose** appropriate initialization for different activation functions

---

## 🎼 The Orchestra Tuning Analogy

**Before a Concert:**
- **Random tuning:** Each musician randomly tunes their instrument
- **Result:** Cacophony, no harmony, terrible performance

**Proper Tuning:**
- **Concert master:** Sets the standard (A = 440 Hz)
- **Each instrument:** Tunes relative to the standard
- **Result:** Beautiful harmony, excellent performance

**In Neural Networks:**
- **Random weights:** Like random tuning → poor gradient flow
- **Proper initialization:** Like proper tuning → excellent training

---

## 💻 Mathematical Foundation

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

print(f"TensorFlow version: {tf.__version__}")
print(f"NumPy version: {np.__version__}")

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Configure plotting
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (14, 10)

In [ ]:
def xavier_initialization():
    """Demonstrate Xavier/Glorot initialization principles"""
    
    print("🧮 XAVIER/GLOROT INITIALIZATION ANALYSIS")
    print("=" * 50)
    
    # Mathematical foundation
    def calculate_xavier_variance(n_in, n_out):
        """Calculate optimal variance for Xavier initialization"""
        return 2.0 / (n_in + n_out)

    def calculate_he_variance(n_in):
        """Calculate optimal variance for He initialization"""
        return 2.0 / n_in
    
    # Test different layer sizes
    layer_configs = [
        (784, 512),   # Input to first hidden
        (512, 256),   # Hidden layer
        (256, 128),   # Hidden layer
        (128, 64),    # Hidden layer
        (64, 10)      # Output layer
    ]
    
    print("📊 OPTIMAL VARIANCE CALCULATIONS:")
    print()
    print(f"{'Layer':<15} {'Input':<8} {'Output':<8} {'Xavier Var':<12} {'He Var':<12} {'Xavier Std':<12} {'He Std':<12}")
    print("-" * 85)
    
    for i, (n_in, n_out) in enumerate(layer_configs, 1):
        xavier_var = calculate_xavier_variance(n_in, n_out)
        he_var = calculate_he_variance(n_in)
        xavier_std = np.sqrt(xavier_var)
        he_std = np.sqrt(he_var)
        
        print(f"Layer {i:<8} {n_in:<8} {n_out:<8} {xavier_var:<12.6f} {he_var:<12.6f} {xavier_std:<12.6f} {he_std:<12.6f}")
    
    print()
    print("💡 KEY INSIGHTS:")
    print("1. Xavier variance = 2/(n_in + n_out) - balances input and output")
    print("2. He variance = 2/n_in - optimized for ReLU activations")
    print("3. Larger input dimensions → smaller initial weights")
    print("4. Proper scaling prevents activation/gradient saturation")
    
    return layer_configs

# Run Xavier initialization analysis
layer_configs = xavier_initialization()

In [ ]:
def compare_initialization_methods():
    """Compare different weight initialization strategies"""
    
    print("\n🔬 INITIALIZATION METHODS COMPARISON")
    print("=" * 50)
    
    # Define initialization strategies
    initializers = {
        'Random Normal (Bad)': tf.keras.initializers.RandomNormal(stddev=1.0),
        'Random Small': tf.keras.initializers.RandomNormal(stddev=0.01),
        'Xavier/Glorot Uniform': tf.keras.initializers.GlorotUniform(),
        'Xavier/Glorot Normal': tf.keras.initializers.GlorotNormal(),
        'He Uniform': tf.keras.initializers.HeUniform(),
        'He Normal': tf.keras.initializers.HeNormal()
    }
    
    # Create models with different initializations
    models = {}
    
    for name, initializer in initializers.items():
        if 'He' in name:
            # Use ReLU for He initialization
            activation = 'relu'
        else:
            # Use tanh for Xavier initialization
            activation = 'tanh'
        
        model = tf.keras.Sequential([
            tf.keras.layers.Dense(128, activation=activation, input_shape=(784,),
                                 kernel_initializer=initializer, name=f'layer1_{name}'),
            tf.keras.layers.Dense(64, activation=activation,
                                 kernel_initializer=initializer, name=f'layer2_{name}'),
            tf.keras.layers.Dense(32, activation=activation,
                                 kernel_initializer=initializer, name=f'layer3_{name}'),
            tf.keras.layers.Dense(16, activation=activation,
                                 kernel_initializer=initializer, name=f'layer4_{name}'),
            tf.keras.layers.Dense(10, activation='softmax', name=f'output_{name}')
        ], name=name.replace(' ', '_').replace('/', '_'))
        
        models[name] = model
    
    print(f"✅ Created {len(models)} models with different initializations")
    return models

# Create models for comparison
comparison_models = compare_initialization_methods()

In [ ]:
def analyze_weight_distributions(models):
    """Analyze initial weight distributions for different initialization methods"""
    
    print("\n📊 WEIGHT DISTRIBUTION ANALYSIS")
    print("=" * 50)
    
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.flatten()
    
    colors = plt.cm.Set3(np.linspace(0, 1, len(models)))
    
    for idx, (name, model) in enumerate(models.items()):
        ax = axes[idx]
        
        # Collect all weights from the model
        all_weights = []
        for layer in model.layers:
            if hasattr(layer, 'kernel'):
                weights = layer.kernel.numpy().flatten()
                all_weights.extend(weights)
        
        all_weights = np.array(all_weights)
        
        # Plot histogram
        ax.hist(all_weights, bins=50, alpha=0.7, color=colors[idx], 
                density=True, label=name)
        
        # Calculate statistics
        mean_weight = np.mean(all_weights)
        std_weight = np.std(all_weights)
        min_weight = np.min(all_weights)
        max_weight = np.max(all_weights)
        
        # Add statistical information
        ax.axvline(mean_weight, color='red', linestyle='--', alpha=0.8, label=f'Mean: {mean_weight:.4f}')
        ax.axvline(mean_weight + std_weight, color='orange', linestyle=':', alpha=0.6)
        ax.axvline(mean_weight - std_weight, color='orange', linestyle=':', alpha=0.6, label=f'±1σ: {std_weight:.4f}')
        
        ax.set_title(f'{name}\nRange: [{min_weight:.3f}, {max_weight:.3f}]', fontsize=10)
        ax.set_xlabel('Weight Value')
        ax.set_ylabel('Density')
        ax.legend(fontsize=8)
        ax.grid(True, alpha=0.3)
        
        # Print statistics
        print(f"\n{name}:")
        print(f"  Mean: {mean_weight:.6f}")
        print(f"  Std:  {std_weight:.6f}")
        print(f"  Range: [{min_weight:.6f}, {max_weight:.6f}]")
        print(f"  Total weights: {len(all_weights)}")
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 DISTRIBUTION INSIGHTS:")
    print("✅ Good initialization: Zero mean, appropriate variance")
    print("⚠️ Too large: May cause saturation or explosion")
    print("⚠️ Too small: May cause vanishing gradients")

# Analyze weight distributions
analyze_weight_distributions(comparison_models)

In [ ]:
def test_gradient_flow_by_initialization(models):
    """Test gradient flow characteristics for different initializations"""
    
    print("\n🌊 GRADIENT FLOW ANALYSIS BY INITIALIZATION")
    print("=" * 60)
    
    # Generate test data
    X_test = tf.random.normal((100, 784))
    y_test = tf.random.uniform((100, 10))
    
    results = {}
    
    for name, model in models.items():
        print(f"\nTesting: {name}")
        
        # Calculate gradients
        with tf.GradientTape() as tape:
            predictions = model(X_test)
            loss = tf.reduce_mean(tf.square(predictions - y_test))
        
        gradients = tape.gradient(loss, model.trainable_variables)
        
        # Analyze gradient characteristics
        grad_norms = []
        for i, grad in enumerate(gradients):
            if grad is not None and i % 2 == 0:  # Only weights, skip biases
                norm = tf.norm(grad).numpy()
                grad_norms.append(norm)
        
        # Calculate metrics
        min_grad = min(grad_norms) if grad_norms else 0
        max_grad = max(grad_norms) if grad_norms else 0
        mean_grad = np.mean(grad_norms) if grad_norms else 0
        std_grad = np.std(grad_norms) if grad_norms else 0
        
        vanished_layers = sum(1 for g in grad_norms if g < 1e-6)
        weak_layers = sum(1 for g in grad_norms if g < 1e-4)
        
        results[name] = {
            'gradient_norms': grad_norms,
            'min_gradient': min_grad,
            'max_gradient': max_grad,
            'mean_gradient': mean_grad,
            'std_gradient': std_grad,
            'vanished_layers': vanished_layers,
            'weak_layers': weak_layers,
            'loss': loss.numpy(),
            'total_layers': len(grad_norms)
        }
        
        print(f"  Gradient range: {min_grad:.2e} to {max_grad:.2e}")
        print(f"  Vanished layers: {vanished_layers}/{len(grad_norms)}")
        print(f"  Weak layers: {weak_layers}/{len(grad_norms)}")
        print(f"  Loss: {loss:.6f}")
    
    return results

# Test gradient flow
gradient_results = test_gradient_flow_by_initialization(comparison_models)

In [ ]:
# Visualize gradient flow comparison
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

init_names = list(gradient_results.keys())
colors = plt.cm.tab10(np.linspace(0, 1, len(init_names)))

# Plot 1: Gradient magnitudes by layer
ax1 = axes[0, 0]
for i, (name, result) in enumerate(gradient_results.items()):
    grad_norms = result['gradient_norms']
    layers = list(range(1, len(grad_norms) + 1))
    ax1.plot(layers, grad_norms, 'o-', color=colors[i], 
             label=name.replace(' ', '\n'), linewidth=2, markersize=6)

ax1.set_yscale('log')
ax1.axhline(y=1e-6, color='red', linestyle='--', alpha=0.5, label='Vanishing threshold')
ax1.axhline(y=1e-4, color='orange', linestyle='--', alpha=0.5, label='Weak threshold')
ax1.set_xlabel('Layer Number')
ax1.set_ylabel('Gradient Magnitude (log scale)')
ax1.set_title('Gradient Flow by Initialization Method')
ax1.legend(fontsize=8, loc='upper right')
ax1.grid(True, alpha=0.3)

# Plot 2: Vanished layers comparison
ax2 = axes[0, 1]
vanished_counts = [gradient_results[name]['vanished_layers'] for name in init_names]
bars = ax2.bar(range(len(init_names)), vanished_counts, color=colors, alpha=0.7)
ax2.set_ylabel('Number of Vanished Layers')
ax2.set_title('Vanished Layers by Initialization')
ax2.set_xticks(range(len(init_names)))
ax2.set_xticklabels([name.replace(' ', '\n') for name in init_names], 
                    rotation=45, ha='right', fontsize=8)

# Add value labels
for bar, value in zip(bars, vanished_counts):
    ax2.text(bar.get_x() + bar.get_width()/2, value + 0.05, str(value), 
             ha='center', va='bottom', fontweight='bold')

# Plot 3: Gradient statistics
ax3 = axes[1, 0]
min_grads = [gradient_results[name]['min_gradient'] for name in init_names]
max_grads = [gradient_results[name]['max_gradient'] for name in init_names]
mean_grads = [gradient_results[name]['mean_gradient'] for name in init_names]

x_pos = np.arange(len(init_names))
width = 0.25

ax3.bar(x_pos - width, min_grads, width, label='Min', alpha=0.7, color='blue')
ax3.bar(x_pos, mean_grads, width, label='Mean', alpha=0.7, color='green')
ax3.bar(x_pos + width, max_grads, width, label='Max', alpha=0.7, color='red')

ax3.set_yscale('log')
ax3.set_ylabel('Gradient Magnitude (log scale)')
ax3.set_title('Gradient Statistics by Initialization')
ax3.set_xticks(x_pos)
ax3.set_xticklabels([name.replace(' ', '\n') for name in init_names], 
                    rotation=45, ha='right', fontsize=8)
ax3.legend()
ax3.grid(True, alpha=0.3)

# Plot 4: Health score comparison
ax4 = axes[1, 1]

# Calculate health scores
health_scores = []
for name in init_names:
    result = gradient_results[name]
    score = 0
    
    # No vanished layers: +3 points
    if result['vanished_layers'] == 0:
        score += 3
    elif result['vanished_layers'] <= 1:
        score += 1
    
    # Good gradient range: +3 points
    if result['min_gradient'] > 1e-5:
        score += 3
    elif result['min_gradient'] > 1e-6:
        score += 1
    
    # No explosion: +2 points
    if result['max_gradient'] < 10:
        score += 2
    elif result['max_gradient'] < 100:
        score += 1
    
    # Stable gradients: +2 points
    if result['std_gradient'] < result['mean_gradient']:
        score += 2
    
    health_scores.append(score)

bars = ax4.bar(range(len(init_names)), health_scores, color=colors, alpha=0.7)
ax4.set_ylabel('Health Score (0-10)')
ax4.set_title('Overall Initialization Health Score')
ax4.set_xticks(range(len(init_names)))
ax4.set_xticklabels([name.replace(' ', '\n') for name in init_names], 
                    rotation=45, ha='right', fontsize=8)
ax4.set_ylim(0, 10)

# Add score labels
for bar, score in zip(bars, health_scores):
    ax4.text(bar.get_x() + bar.get_width()/2, score + 0.1, f'{score}/10', 
             ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
def lsuv_initialization(model, X_sample, target_var=1.0, max_iterations=10):
    """Layer-Sequential Unit-Variance initialization implementation"""

    print("\n🎯 LSUV INITIALIZATION DEMONSTRATION")
    print("=" * 50)
    print("LSUV: Layer-Sequential Unit-Variance Initialization")
    print("Goal: Each layer output has unit variance")
    print()

    # Create a fresh model for LSUV
    lsuv_model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(784,), name='layer1'),
        tf.keras.layers.Dense(64, activation='relu', name='layer2'),
        tf.keras.layers.Dense(32, activation='relu', name='layer3'),
        tf.keras.layers.Dense(16, activation='relu', name='layer4'),
        tf.keras.layers.Dense(10, activation='softmax', name='output')
    ], name='LSUV_Model')

    print("📊 LSUV Calibration Process:")
    print("=" * 40)

    activation_variances = []
    layer_adjustments = []

    # Process each layer sequentially
    for i, layer in enumerate(lsuv_model.layers[:-1]):  # Skip output layer
        if hasattr(layer, 'kernel'):  # Only for Dense layers
            print(f"\n🔧 Calibrating Layer {i+1} ({layer.name})...")
            
            layer_variances = []
            adjustments = 0

            for iteration in range(max_iterations):
                # Get current activations up to this layer
                temp_model = tf.keras.Sequential(lsuv_model.layers[:i+1])
                activations = temp_model(X_sample)

                # Calculate variance
                current_var = tf.reduce_mean(tf.square(activations)).numpy()
                layer_variances.append(current_var)

                print(f"  Iteration {iteration+1}: Variance = {current_var:.6f}")

                # Check convergence
                if abs(current_var - target_var) < 0.01:
                    print(f"  ✅ Converged to target variance ({target_var})!")
                    break

                # Adjust weights to achieve target variance
                if current_var > 0:  # Avoid division by zero
                    scale_factor = np.sqrt(target_var / current_var)
                    layer.kernel.assign(layer.kernel * scale_factor)
                    adjustments += 1
                    print(f"  🔄 Applied scaling factor: {scale_factor:.4f}")
                else:
                    print(f"  ⚠️ Zero variance detected, skipping adjustment")
                    break

            activation_variances.append(layer_variances)
            layer_adjustments.append(adjustments)
            print(f"  📋 Total adjustments for Layer {i+1}: {adjustments}")

    print("\n✅ LSUV initialization complete!")
    
    # Visualize the calibration process
    fig, axes = plt.subplots(1, 2, figsize=(15, 6))
    
    # Plot 1: Variance evolution
    ax1 = axes[0]
    for i, variances in enumerate(activation_variances):
        iterations = list(range(1, len(variances) + 1))
        ax1.plot(iterations, variances, 'o-', label=f'Layer {i+1}', linewidth=2, markersize=6)
    
    ax1.axhline(y=target_var, color='red', linestyle='--', alpha=0.7, 
                label=f'Target Variance ({target_var})')
    ax1.set_xlabel('Iteration')
    ax1.set_ylabel('Activation Variance')
    ax1.set_title('LSUV Variance Calibration Process')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Number of adjustments per layer
    ax2 = axes[1]
    layers = [f'Layer {i+1}' for i in range(len(layer_adjustments))]
    bars = ax2.bar(layers, layer_adjustments, color='skyblue', alpha=0.7)
    ax2.set_ylabel('Number of Adjustments')
    ax2.set_title('LSUV Adjustments per Layer')
    
    # Add value labels
    for bar, value in zip(bars, layer_adjustments):
        ax2.text(bar.get_x() + bar.get_width()/2, value + 0.05, str(value), 
                 ha='center', va='bottom', fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    return lsuv_model

# Demonstrate LSUV initialization
X_sample = tf.random.normal((1000, 784))
lsuv_model = lsuv_initialization(None, X_sample)

In [ ]:
# Compare LSUV with other methods
print("\n⚔️ LSUV vs OTHER INITIALIZATION METHODS")
print("=" * 50)

# Test LSUV model
X_test = tf.random.normal((100, 784))
y_test = tf.random.uniform((100, 10))

with tf.GradientTape() as tape:
    predictions = lsuv_model(X_test)
    loss = tf.reduce_mean(tf.square(predictions - y_test))

gradients = tape.gradient(loss, lsuv_model.trainable_variables)
grad_norms = [tf.norm(g).numpy() for i, g in enumerate(gradients) if g is not None and i % 2 == 0]

lsuv_result = {
    'gradient_norms': grad_norms,
    'min_gradient': min(grad_norms),
    'max_gradient': max(grad_norms),
    'mean_gradient': np.mean(grad_norms),
    'vanished_layers': sum(1 for g in grad_norms if g < 1e-6),
    'weak_layers': sum(1 for g in grad_norms if g < 1e-4)
}

# Add LSUV to comparison
gradient_results['LSUV'] = lsuv_result

# Create final comparison table
print("\n📊 FINAL INITIALIZATION COMPARISON")
print("=" * 80)
print(f"{'Method':<25} {'Min Grad':<12} {'Max Grad':<12} {'Mean Grad':<12} {'Vanished':<10} {'Health':<8}")
print("-" * 80)

for name, result in gradient_results.items():
    # Calculate health score
    score = 0
    if result['vanished_layers'] == 0: score += 3
    if result['min_gradient'] > 1e-5: score += 3
    if result['max_gradient'] < 10: score += 2
    if result.get('std_gradient', 0) < result['mean_gradient']: score += 2
    
    health_emoji = "🟢" if score >= 7 else "🟡" if score >= 5 else "🟠" if score >= 3 else "🔴"
    
    print(f"{name:<25} {result['min_gradient']:<12.2e} {result['max_gradient']:<12.2e} "
          f"{result['mean_gradient']:<12.2e} {result['vanished_layers']:<10} {health_emoji} {score}/10")

print("\n🏆 WINNER: The method with the highest health score!")
best_method = max(gradient_results.keys(), 
                 key=lambda x: 3*(gradient_results[x]['vanished_layers']==0) + 
                              3*(gradient_results[x]['min_gradient']>1e-5) + 
                              2*(gradient_results[x]['max_gradient']<10))
print(f"🎖️ Best performing initialization: {best_method}")

---

## 🔍 Initialization Strategy Guide

### 📊 Method Comparison Summary

| Method | Best For | Pros | Cons | Health Score |
|--------|----------|------|------|-------------|
| **He Initialization** | ReLU networks | Prevents vanishing, simple | Only for ReLU | 🟢 8-9/10 |
| **Xavier/Glorot** | Sigmoid/Tanh | Mathematical foundation | Poor for ReLU | 🟡 6-7/10 |
| **LSUV** | Any activation | Adaptive, robust | Computational overhead | 🟢 8-10/10 |
| **Random Normal (small)** | Shallow networks | Simple | Poor for deep networks | 🟠 4-5/10 |
| **Random Normal (large)** | None | None | Causes explosions | 🔴 1-2/10 |

### 🎯 **Practical Guidelines**

#### ✅ **Use He Initialization when:**
- Using ReLU or ReLU variants (Leaky ReLU, ELU)
- Building deep networks (>5 layers)
- Need simple, reliable performance
- Computational efficiency is important

#### ✅ **Use Xavier/Glorot when:**
- Using sigmoid or tanh activations
- Building shallow to medium networks (<10 layers)
- Following classical approaches

#### ✅ **Use LSUV when:**
- Building very deep networks (>20 layers)
- Using mixed activation functions
- Need maximum performance
- Can afford initialization overhead

#### ❌ **Avoid:**
- Large random initialization (stddev > 1.0)
- Zero initialization (no gradients)
- Same initialization for all layer types
- Ignoring activation function choice

---

## 💡 Key Mathematical Insights

### **Xavier Initialization Formula:**
```
Variance = 2 / (n_input + n_output)
```
**Reasoning:** Balances forward and backward pass variance

### **He Initialization Formula:**
```
Variance = 2 / n_input
```
**Reasoning:** Accounts for ReLU killing half the activations

### **LSUV Principle:**
```
For each layer: Scale weights until Var(activations) = 1
```
**Reasoning:** Ensures optimal activation magnitude layer by layer

---

## 🧪 Experimental Findings

From our experiments:

1. **He initialization** consistently produces healthy gradient flow for ReLU networks
2. **Large random weights** (stddev=1.0) cause severe gradient problems
3. **LSUV** adapts to any architecture but requires more computation
4. **Proper initialization** can be more important than architecture choice
5. **Activation function** determines optimal initialization strategy

---

## 🎯 Next Steps

In the next notebook, we'll explore:
- **Batch normalization** implementation and theory
- **How normalization solves initialization problems**
- **LayerNorm vs BatchNorm** comparisons

---

*This notebook demonstrates Concept 5 of Week 5: Deep Neural Network Architectures*